In [ ]:
!pip install torch==2.1.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.1.0+cu121
# !pip install --upgrade sympy
# !pip uninstall -y transformers
# !pip install transformers
!pip install bitsandbytes

ERROR: Could not find a version that satisfies the requirement torch==2.1.0+cu121 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==2.1.0+cu121


In [ ]:
from google.colab import drive
import numpy as np
from huggingface_hub import notebook_login
import torch
import time
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import os
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
notebook_login()

In [ ]:
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/PW_NLP

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/PW_NLP


In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

!nvidia-smi

12.4
True
Sun Feb  2 18:15:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------------------

# Mistral

In [ ]:
model_card = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_card)
tokenizer.pad_token = tokenizer.eos_token

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

PackageNotFoundError: No package metadata was found for bitsandbytes

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.temperature = 0.0
generation_config.num_return_sequences = 1

## 2 prompts: simple rephrasing and Q2D

In [ ]:
prompt_rephrase = """You are a fincance expert, with the abilty of rewriting query.
Rephrase a query so that it contains more information, use synonyms and your knowledge to understand the possible context.
Read carefully the query, understand the concept that is asked and explain it. If acronyms are used, expand and explain their meaning.

This is an example query:
query: 'what is the aaa interest rate in 2009?'

This is the corresponding query expanded:
query expanded: 'What was the interest rate for bonds or debt instruments rated AAA (the highest credit rating assigned by agencies such as Standard & Poor’s, Moody’s, or Fitch) in the year 2009? Specifically, I am looking for the yield or average interest rate on AAA-rated corporate or government bonds during that period.'

Return the query expanded in a dictionary format:
{
  "query": "",
  "answer": ""
}

Your input query:
"""

prompt_Q2D = """You are a fincance expert.
Write a passage that answers the given query, if you find acronyms, expand them and explain them.
If there are specific finance concepts in the query, try to explain them briefly.
Don't be too verbose, just one paragraph.

This is an example query:
query: 'what is the aaa interest rate in 2009?'

This is the corresponding query answer paragraph:
paragraph: 'The AAA interest rate in 2009 refers to the interest rate associated with bonds that were rated AAA, which is the highest credit rating assigned by credit rating agencies like Standard & Poor's, Moody's, and Fitch. AAA-rated bonds are considered to have the lowest risk of default and, therefore, typically offer lower yields compared to bonds with lower ratings. In 2009, due to the global financial crisis, interest rates were generally lower as central banks, such as the U.S. Federal Reserve, slashed rates to stimulate economic growth.'

Return the query paragraph in a dictionary format:
{
  "query": "",
  "answer": ""
}
Your input query:
"""

In [ ]:
def call_model(model, prompt_tokenized, generation_config, tokenizer):
    start_time = time.time()
    with torch.no_grad():
      output = model.generate(
          **prompt_tokenized,
          max_new_tokens=generation_config.max_new_tokens,
          eos_token_id=generation_config.eos_token_id,
          pad_token_id=generation_config.pad_token_id,
          temperature=generation_config.temperature,
          num_return_sequences=generation_config.num_return_sequences
      )
      end_time = time.time()

    # generated_text = tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text, round(end_time-start_time, 2)

## Parsing of the response
The models doesn't always follow the instruction, so various changes were done to this function

In [ ]:
def parse_response_1(response: str) -> str:
    match = re.search(r'(Your output answer:|Your expanded query:|The query answer paragraph:|The query answer paragraph:|The expanded query would be:|Your output query:|Expanded query:|Your query answer paragraph:|Your output query paragraph:|The answer paragraph:|Your query expanded:|The corresponding query answer paragraph:|Your answer:)\s*(.*)', response, re.DOTALL)
    return match.group(2).strip() if match else "Error in parsing"

def extreme_parsing(text):
    # Define regex pattern to capture the last "answer" value
    answer_pattern = r'"answer":\s*"([^"]+)"\s*(?:,|\})[^}]*$'

    # Extract the last answer using regex
    answer_match = re.search(answer_pattern, text)
    if answer_match:
        answer = answer_match.group(1)
        return answer
    else:
        return parse_response_1(text)
        # return "Error in parsing"

## Actual function that compute the expansion for each dataset

In [ ]:
def expanding_queries(dataset, prompt_template, template_name, max_new_tokens=1000):
    generation_config.max_new_tokens = max_new_tokens
    print(f"EXPANDING QUERIES FOR {dataset}, with template: {prompt_template}\n\n")

    # Check if the results file are already there, and continue in case
    if os.path.exists(f'queries_expanded_{dataset}_{template_name}.npy'):
        queries_expanded_npy = np.load(f'queries_expanded_{dataset}_{template_name}.npy', allow_pickle='TRUE').item()
        print(f"queries_expanded_{dataset}_{template_name}.npy already created!\t It contains: ", len(queries_expanded_npy), " queries expanded ", list(queries_expanded_npy.keys()))
    else:
        queries_expanded_npy = {}
    if os.path.exists(f'Timing_queries_{dataset}_{template_name}.npy'):
        timing_npy = np.load(f'Timing_queries_{dataset}_{template_name}.npy', allow_pickle='TRUE').item()
        print(f"Timing_queries_{dataset}_{template_name}.npy already created!\t It contains: ", len(timing_npy), " queries expanded ")
    else:
        timing_npy = {}

    # Load the extracted queries to compute the queries
    query_dict = np.load(f'extracted_queries_{dataset}.npy', allow_pickle='TRUE').item()
    print(f"\nLoaded extracted_queries_{dataset}, it has {len(query_dict)} queries to process")

    count = len(queries_expanded_npy)
    print("\n\n")
    for key, value in query_dict.items():
        count += 1
        print(f"\t{count} - Key: {key}")
        # Check if the query has already been processed
        if key in queries_expanded_npy:
            print(f"\t\tSkipping {key}, already processed ...\n")
            continue
        if value == []:
            print(f"\t\tSkipping {key}, no query found ...\n")
            continue

        query = value
        print("query: ", query)
        prompt = prompt_template + query
        # Tokenize the prompt
        input_ids_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Call the model
        model_response, total_time = call_model(model, input_ids_prompt, generation_config, tokenizer)
        # Parse the response
        parsed_resp = parse_response(model_response)
        if parsed_resp == "Error in parsing" or parsed_resp=="":
          print("MODEL RESPONSE", model_response)
        print("-->: \t", parsed_resp)

        print(f"\tTime: {total_time} seconds")
        print("\n")
        queries_expanded_npy[key] = parsed_resp
        np.save(f'queries_expanded_{dataset}_{template_name}.npy', queries_expanded_npy)
        timing_npy[key] = total_time
        np.save(f'Timing_queries_{dataset}_{template_name}.npy', timing_npy)

In [ ]:
dataset_names = ['ConvFinQA', 'FinQA', 'MultiHeritt', 'TATQA']
print(dataset_names)
for DATASET_NAME in dataset_names:
    print("================== DATASET:", DATASET_NAME, " ==================\n")

    query_to_extract = np.load(f'extracted_queries_{DATASET_NAME}.npy', allow_pickle='TRUE').item()
    print(f"DATASET: {DATASET_NAME} - {len(query_to_extract)} queries to expand")

    template_name = "rephrase"
    print("========= PROMPT: ", template_name, "=========n")
    expanding_queries(DATASET_NAME, prompt_rephrase, template_name, max_new_tokens=1000)
    query_dict = np.load(f'queries_expanded_{DATASET_NAME}_{template_name}.npy', allow_pickle='TRUE').item()
    print(f"FINISHHH: expanded {len(query_dict)} queries with prompt: {template_name}")
    print()

    template_name = "Q2D"
    print("========= PROMPT: ", template_name, "=========n")
    expanding_queries(DATASET_NAME, prompt_Q2D, template_name, max_new_tokens=2000)
    query_dict = np.load(f'queries_expanded_{DATASET_NAME}_{template_name}.npy', allow_pickle='TRUE').item()
    print(f"FINISHHH: expanded {len(query_dict)} queries with prompt: {template_name}")
    print()

['ConvFinQA', 'FinQA', 'MultiHeritt', 'TATQA']
================== DATASET: ConvFinQA  ==================

DATASET: ConvFinQA - 126 queries to expand
========= PROMPT:  rephrase =========n
EXPANDING QUERIES FOR ConvFinQA, with template: You are a fincance expert, with the abilty of rewriting query.
Rephrase a query so that it contains more information, use synonyms and your knowledge to understand the possible context.
Read carefully the query, understand the concept that is asked and explain it. If acronyms are used, expand and explain their meaning.

This is an example query:
query: 'what is the aaa interest rate in 2009?'

This is the corresponding query expanded:
query expanded: 'What was the interest rate for bonds or debt instruments rated AAA (the highest credit rating assigned by agencies such as Standard & Poor’s, Moody’s, or Fitch) in the year 2009? Specifically, I am looking for the yield or average interest rate on AAA-rated corporate or government bonds during that period.'

## Adjust error in parsing the response

In [ ]:
dataset_names = ['ConvFinQA', 'FinQA', 'MultiHeritt', 'TATQA']
for DATASET_NAME in dataset_names:
    template_name = "rephrase"

    prompt_rephrase = f"""You are a fincance expert, with the abilty of rewriting query.
Rephrase a query so that it contains more information, use synonyms and your knowledge to understand the possible context.
Read carefully the query, understand the concept that is asked and explain it. If acronyms are used, expand and explain their meaning.

This is an example query:
query: 'what is the aaa interest rate in 2009?'

This is the corresponding query expanded:
query expanded: 'What was the interest rate for bonds or debt instruments rated AAA (the highest credit rating assigned by agencies such as Standard & Poor’s, Moody’s, or Fitch) in the year 2009? Specifically, I am looking for the yield or average interest rate on AAA-rated corporate or government bonds during that period.'

Return the query expanded.
Your input query:
"""

    expanded_dict = np.load(f'queries_expanded_{DATASET_NAME}_{template_name}.npy', allow_pickle='TRUE').item()
    print(f"DATASET: {DATASET_NAME} - {len(expanded_dict)} queries expanded")

    query_dict = np.load(f'extracted_queries_{DATASET_NAME}.npy', allow_pickle='TRUE').item()
    print(f"\nLoaded extracted_queries_{DATASET_NAME}, it has {len(query_dict)} queries to process")

    count = 0
    for key, value in expanded_dict.items():
        if value == "Error in parsing":
            count += 1
            query = query_dict[key]
            print(f"query again id {key}:\t{query}")

            prompt = prompt_rephrase + query

            input_ids_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)

            model_response, total_time = call_model(model, input_ids_prompt, generation_config, tokenizer)

            # Parse the response
            parsed_resp = parse_response(model_response)
            if parsed_resp == "Error in parsing":
                print("MODEL RESPONSE\n", model_response)
            print("--->: \t", parsed_resp)
            print(f"\tTime: {total_time} seconds")
            print("\n")

            expanded_dict[key] = parsed_resp
            np.save(f'queries_expanded_{DATASET_NAME}_{template_name}.npy', expanded_dict)

    print(f"============ DATASET: {DATASET_NAME} - {count} queries with error parsing ============\n")


DATASET: ConvFinQA - 126 queries expanded

Loaded extracted_queries_ConvFinQA, it has 126 queries to process
============ DATASET: ConvFinQA - 0 queries with error parsing ============

DATASET: FinQA - 344 queries expanded

Loaded extracted_queries_FinQA, it has 344 queries to process
============ DATASET: FinQA - 0 queries with error parsing ============

DATASET: MultiHeritt - 292 queries expanded

Loaded extracted_queries_MultiHeritt, it has 292 queries to process
============ DATASET: MultiHeritt - 0 queries with error parsing ============

DATASET: TATQA - 498 queries expanded

Loaded extracted_queries_TATQA, it has 498 queries to process
============ DATASET: TATQA - 0 queries with error parsing ============



### Adjust for Q2D

In [ ]:
dataset_names = ['ConvFinQA', 'FinQA', 'MultiHeritt', 'TATQA']
for DATASET_NAME in dataset_names:
    template_name = "Q2D"

    prompt_Q2D = """You are a fincance expert.
Write a passage that answers the given query, if you find acronyms, expand them and explain them.
If there are specific finance concepts in the query, try to explain them briefly.
Don't be too verbose, just one paragraph.

This is an example query:
query: 'what is the aaa interest rate in 2009?'

This is the corresponding query answer paragraph:
paragraph: 'The AAA interest rate in 2009 refers to the interest rate associated with bonds that were rated AAA, which is the highest credit rating assigned by credit rating agencies like Standard & Poor's, Moody's, and Fitch. AAA-rated bonds are considered to have the lowest risk of default and, therefore, typically offer lower yields compared to bonds with lower ratings. In 2009, due to the global financial crisis, interest rates were generally lower as central banks, such as the U.S. Federal Reserve, slashed rates to stimulate economic growth.'

Return the query paragraph in a dictionary format:
{
  "query": "",
  "answer": ""
}
Your input query:
"""

    expanded_dict = np.load(f'queries_expanded_{DATASET_NAME}_{template_name}.npy', allow_pickle='TRUE').item()
    print(f"DATASET: {DATASET_NAME} - {len(query_dict)} queries expanded")

    if os.path.exists(f'Timing_queries_{DATASET_NAME}_{template_name}.npy'):
        timing_npy = np.load(f'Timing_queries_{DATASET_NAME}_{template_name}.npy', allow_pickle='TRUE').item()

    query_dict = np.load(f'extracted_queries_{DATASET_NAME}.npy', allow_pickle='TRUE').item()
    print(f"\nLoaded extracted_queries_{DATASET_NAME}, it has {len(query_dict)} queries to process")

    count = 0
    for key, value in expanded_dict.items():
        if value == "Error in parsing":
            count += 1
    print(f"DATASET: {DATASET_NAME} - {count} queries with error parsing\n")

    qs = ['q6169977e', 'q616ab744', 'q616a61d6', 'q61666c52', 'q6166708a']
    count = 0
    for key, value in expanded_dict.items():
        if key in qs:
            count += 1
            query = query_dict[key]
            print(f"query again id {key}:\t{query}")

            prompt = prompt_Q2D + query

            input_ids_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)

            model_response, total_time = call_model(model, input_ids_prompt, generation_config, tokenizer)

            # Parse the response
            parsed_resp = parse_response(model_response)
            if parsed_resp == "Error in parsing":
                print("MODEL RESPONSE", model_response)
            print("-->: \t", parsed_resp)
            print(f"\tTime: {total_time} seconds")
            print("\n")

            expanded_dict[key] = parsed_resp
            np.save(f'queries_expanded_{DATASET_NAME}_{template_name}.npy', expanded_dict)
            timing_npy[key] = total_time
            np.save(f'Timing_queries_{DATASET_NAME}_{template_name}.npy', timing_npy)

    print(f"finish DATASET: {DATASET_NAME} - with {count} queries with error parsing\n\n")


# Trial with FLAN T5

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Choose a model that fits within your GPU memory
model_name = "google/flan-t5-xxl"
# Change to:
# 'flan-t5-small':     77M
# 'flan-t5-base':      248M
# 'flan-t5-large':     783M
# 'flan-t5-xl':        2.85B
# 'flan-t5-xxl':       11.3B

# Load model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

example_query = "what is the aaa interest rate in 2009?"
example_query_expanded = "What was the interest rate for bonds or debt instruments rated AAA (the highest credit rating assigned by agencies such as Standard & Poor’s, Moody’s, or Fitch) in the year 2009? Specifically, I am looking for the yield or average interest rate on AAA-rated corporate or government bonds during that period."

query = "what is the assumed fmv of a share?"

input_text = f"""You are a fincance expert, with the abilty of rewriting query.
Rephrase a query so that it contains more information, use synonyms and your knowledge to understand the possible context.
Read carefully the query, understand the concept that is asked and explain it. If acronyms are used, expand and explain their meaning.

This is an example query: {example_query}
This is the corresponding query expanded: {example_query_expanded}

Your input query: {query}
Write the corresponding query expanded. """

inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

with torch.no_grad():
    output_tokens = model.generate(**inputs, max_length=50)
    # outputs = model.generate(input_ids)

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(output_text)

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig

# Choose a Flan-T5 model (large, base, or small)
model_name = "google/flan-t5-large"

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computation
    bnb_4bit_use_double_quant=True,  # Further quantization for reduced memory
)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load model with quantization
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Example inference
input_text = "Translate English to French: How are you?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate output
with torch.no_grad():
    output_tokens = model.generate(**inputs, max_length=50)

# Decode and print result
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(output_text)
